# Benchmarking SurrealDB's feature of executing a model internally

## Table of contents

1. [General dependencies and helpers](#general-dependencies-and-helpers)

## General dependencies and helpers

In [8]:
%pip install "git+https://github.com/surrealdb/surrealml#egg=surrealml[sklearn,torch]"
%pip install -q onnx onnxscript

import subprocess

  Cloning https://github.com/surrealdb/surrealml to /tmp/pip-install-l977ll04/surrealml_69023d38e19e4912b684c3057019787b
  Running command git clone --filter=blob:none --quiet https://github.com/surrealdb/surrealml /tmp/pip-install-l977ll04/surrealml_69023d38e19e4912b684c3057019787b
  Resolved https://github.com/surrealdb/surrealml to commit 9e0b5f6c8c6a0729646104a31721748a1be5b641
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


We aim to time the two cases:
1. **surreal**, i.e. predicting with the model in ONNX format _inside_ the SurrealDB, and then fetching the prediction from SurrealDB.
2. **pytorch**, i.e. fetching the data from SurrealDB and _externally_ predicting with the PyTorch model.

To handle timing in a consistent way, we define the decorator `chronometer`.

In [2]:
import time
from functools import wraps


def chronometer(foo):
    @wraps(foo)
    def wrapper(*args, **kwargs):
        start = time.time()
        _ = foo(*args, *kwargs)
        end = time.time()
        return end - start

    return wrapper

In [3]:
import torch
import torch.nn as nn


class Fnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def __str__(self) -> str:
        return self.__class__.__name__

ModuleNotFoundError: No module named 'torch'

In [ ]:
model = Fnn()
model.load_state_dict(torch.load("parameters.pth")["model_state_dict"])
model.eval()

In [ ]:
from surrealml import SurMlFile, Engine

example_input = torch.rand(1, 10)
surml_file = SurMlFile(
    model=model, name=str(model), inputs=example_input, engine=Engine.PYTORCH
)

path_surml = "./model.surml"
surml_file.add_version("0.0.1")
surml_file.save(path_surml)

In [ ]:
"""
The URL of the SurrealDB instance, and the authentication details, as well as the namespace and database in scope
"""

URL = "http://localhost:8000"
NS = "latency_test"
DB = "surreal_ml_vs_pytorch"
USR = "user"
PASS = "user"
CRD = (USR, PASS)
PATH = "file://surreal"

In [ ]:
"""
The CLI command to import the model in the SurrealDB database of choice, this time we use it programmatically
"""

command = [
    "surreal",
    "ml",
    "import",
    "--endpoint",
    URL,
    "--user",
    USR,
    "--pass",
    PASS,
    "--ns",
    NS,
    "--db",
    DB,
    path_surml,
]


subprocess.Popen(
    command,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
output = subprocess.check_output(command, stderr=subprocess.STDOUT)
output_str = output.decode("utf-8")
print(output_str)

In [ ]:
%%capture
"""
###################################################################################################
--> The other way to upload a model to SurrealDB. Could not make it work by the time of writing :( 
###################################################################################################

surml_file.upload(
    path="./model.surml",
    url=URL,
    chunk_size=36864,
    namespace=NS,
    database=DB,
    username=USR,
    password=PASS)
"""

In [ ]:
"""
The code that was used to generate the test data, and load it in SurrealDB
"""

from surrealist import Surreal

surreal = Surreal(
    url=URL,
    namespace=NS,
    database=DB,
    credentials=CRD,
    log_level="ERROR",
    timeout=10**4,
)

max_test_size = 10**4
# chunk_size should divide max_test_size, we do it as we create the test inputs in memory, and thus we should avoid a memory crash.
chunk_size = 10**2
number_chunks = int(max_test_size / chunk_size)

# likewise, test_step should divide max_test_size
test_step = 10**3
number_steps = int(max_test_size / test_step)

with surreal.connect() as connect:
    for _ in range(number_chunks):
        test_inputs = torch.rand(chunk_size, 10).tolist()
        [
            connect.query(f"CREATE inputs:ulid() SET value = {input};")
            for input in test_inputs
        ]

In [ ]:
surreal_times = []
try:
    with surreal.connect() as connect:
        for increment in range(number_steps):
            test_size = (increment + 1) * test_step

            # in a prior run, the query result was tested with assert query_result["status"] == "OK"
            # also do print(query_result) to check further
            @chronometer
            def evaluate_with_surrealdb():
                _ = connect.query(
                    f"SELECT VALUE ml::Fnn<0.0.1>(value) FROM inputs LIMIT {test_size};"
                ).to_dict()["result"]

            elapsed_time = evaluate_with_surrealdb()
            print(f"For {test_size} datapoints, it took {elapsed_time} seconds")
            surreal_times.append(elapsed_time)
except Exception as e:
    print(e)

In [ ]:
pytorch_times = []
try:
    with surreal.connect() as connect:
        for increment in range(1, 11):
            test_size = increment * test_step

            @chronometer
            def evaluate_pytorch():
                inputs = connect.query(
                    f"SELECT VALUE value FROM inputs LIMIT {test_size}"
                ).to_dict()["result"]

                with torch.no_grad():
                    _ = model.forward(torch.tensor(inputs))

            elapsed_time = evaluate_pytorch()
            print(f"For {test_size} datapoints, it took {elapsed_time} seconds")
            pytorch_times.append(elapsed_time)
except Exception as e:
    print(e)